In [2]:
import numpy as np
import pandas as pd
import pickle
import preprocessing_helper as helper

from pathlib import Path
import constants
import json 
from genson import SchemaBuilder

In [ ]:
# TODO: I believe this notebook is not used as broken

### 1. Get directories related information and load in MoonBoard 2016 data

Load the sccraped data

In [3]:
with open(constants.SCRAPE_DATA_PATH, 'rb') as f:
    MoonBoard_2016_raw = pickle.load(f)

Dump the scraped data to JSON

In [4]:
def dump_to_json():
  with open(constants.DATA_DIR / 'MoonBoard_2016_raw.json', 'w') as f:
    json.dump(MoonBoard_2016_raw, f)

# dump_to_json()

In [16]:
# get first 5 values of MoonBoard_2016_raw
list(MoonBoard_2016_raw.values())[:1]

[{'problem_name': 'TALL POPPY',
  'info': ['halladay',
   'Be the first to repeat this problem',
   '6C+',
   'Feet follow hands',
   ''],
  'url': 'https://moonboard.com/Problems/View/367894/tall-poppy',
  'num_empty': 3,
  'num_stars': 0,
  'moves': [{'Id': 2119224,
    'Description': 'F5',
    'IsStart': True,
    'IsEnd': False},
   {'Id': 2119225, 'Description': 'G2', 'IsStart': True, 'IsEnd': False},
   {'Id': 2119226, 'Description': 'H10', 'IsStart': False, 'IsEnd': False},
   {'Id': 2119227, 'Description': 'B11', 'IsStart': False, 'IsEnd': False},
   {'Id': 2119228, 'Description': 'E15', 'IsStart': False, 'IsEnd': False},
   {'Id': 2119229, 'Description': 'D18', 'IsStart': False, 'IsEnd': True},
   {'Id': 2119230, 'Description': 'E8', 'IsStart': False, 'IsEnd': False}],
  'grade': '6C+',
  'UserGrade': None,
  'isBenchmark': False,
  'repeats': 0,
  'ProblemType': None,
  'IsMaster': False,
  'setter': {'Id': '93A38FE2-1B2B-4B0F-9B70-9A77BAE976B8',
   'Nickname': 'halladay',
  

Generate the schema for the scraped data

In [6]:
def generate_schema():
  builder = SchemaBuilder()
  builder.add_object(list(MoonBoard_2016_raw.values()))
  builder.to_schema()

  with open(constants.DATA_DIR / 'MoonBoard_2016_raw_schema.json', 'w') as f:
    json.dump(builder.to_schema(), f)
  
# generate_schema()

Loads hold difficulties for MoonBoard from excel sheet?

The hold difficulties appear to be defined by a vector of 6 different difficulties. Each element defines the difficulty for?

In [8]:
features = pd.read_excel(constants.HOLD_FEATURES_PATH, dtype=str)

# convert features from pd dataframe to dictionary
feature_dict = {}
for index in features.index:
    feature_item = features.loc[index]
    feature_dict[(int(feature_item['X_coord']), int(feature_item['Y_coord']))] = np.array(
        list(feature_item['Difficulties'])).astype(int)

In [14]:
# TODO: what to the individual difficulties mean?? Consider copilot suggestions or see where used
list(feature_dict.items())[:5]

[((0, 17), array([0, 3, 4, 7, 3, 0])),
 ((0, 15), array([1, 4, 2, 2, 1, 0])),
 ((0, 14), array([0, 1, 3, 1, 0, 0])),
 ((0, 13), array([2, 6, 2, 0, 0, 0])),
 ((0, 12), array([0, 1, 2, 1, 0, 1]))]

### 2. Separate data into different categories:
- no user ratings
- benchmarked with user ratings
- non-benchmarked with user ratings

In [10]:
grade_map = helper.get_grade_map()
grade_map

{'6B': 0,
 '6B+': 0,
 '6C': 1,
 '6C+': 1,
 '7A': 2,
 '7A+': 3,
 '7B': 4,
 '7B+': 4,
 '7C': 5,
 '7C+': 6,
 '8A': 7,
 '8A+': 8,
 '8B': 9}

Processing the raw data is currently broken, as it appears the scraped data has changed since this was written.
Particularly the json for each route has changed.

Originally:
- 'start'/'mid'/'end' contained respective holds for the problem
- each hold is (x,y) coordinate

Current scraped data:
- ```
  'moves': [
    {'Id': 2119224, 'Description': 'F5', 'IsStart': True, 'IsEnd': False},
    {'Id': 2119225, 'Description': 'G2', 'IsStart': True, 'IsEnd': False},
    {'Id': 2119226, 'Description': 'H10', 'IsStart': False, 'IsEnd': False},
    {'Id': 2119227, 'Description': 'B11', 'IsStart': False, 'IsEnd': False},
    {'Id': 2119228, 'Description': 'E15', 'IsStart': False, 'IsEnd': False},
    {'Id': 2119229, 'Description': 'D18', 'IsStart': False, 'IsEnd': True},
    {'Id': 2119230, 'Description': 'E8', 'IsStart': False, 'IsEnd': False}
  ],
  ```

In [32]:
save_path = constants.PREPROCESSING_DIR / 'processed_data_xy_mode'
# TODO: fix see above
output_xy_mode = helper.classify_and_reorganize_data(MoonBoard_2016_raw, save_path, delta_xy_mode = False)

Raw data with key 367894 contains error


KeyError: 'start'

### 3. Data Preprocessing with BetaMove

Load the the data split into dicts:
- `X_dict_benchmark_withgrade`
- `Y_dict_benchmark_withgrade`
- `X_dict_benchmark_nograde`
- `Y_dict_benchmark_nograde`
- `X_dict_withgrade`
- `Y_dict_withgrade`
- `X_dict_nograde`
- `Y_dict_nograde`

Where each X item is (problem_id, 10xn array) where n is the number of holds in the problem. Each **column** vector contain:
- 0: `feature_dict[x,y][0]` 0-9
- 1: `feature_dict[x,y][1]` 0-9
- 2: `feature_dict[x,y][2]` 0-9
- 3: `feature_dict[x,y][3]` 0-9
- 4: `feature_dict[x,y][4]` 0-9
- 5: `feature_dict[x,y][5]` 0-9
- 6: `x` 0-10
- 7: `y` 0-17
- 8: `is_start` 0/1
- 9: `is_end` 0/1

Y items (problem_id, 3x1 array). Each **column** vector contain:
- 0: `grade` 0-9
- 1: `is_benchmarked` 0/1
- 2: `user_grade` (only if dict is `withgrade`) 0/1

In [19]:
# load the data meant to be generated by the previous step
raw_data_forseq_path = constants.PREPROCESSING_DIR / 'processed_data_xy_mode'
with open(raw_data_forseq_path, 'rb') as f:
    MoonBoard_2016_raw_forseq = pickle.load(f)

In [20]:
# this is a single problem x, each column of matrix is a hold as described above
MoonBoard_2016_raw_forseq['X_dict_benchmark_withgrade'][next(iter(MoonBoard_2016_raw_forseq['X_dict_benchmark_withgrade']))]

array([[ 0.,  2.,  0.,  1.,  2.,  3.],
       [ 2.,  5.,  1.,  5.,  3.,  7.],
       [ 5.,  2.,  4.,  3.,  4.,  5.],
       [ 2.,  0.,  1.,  1.,  2.,  2.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [10.,  5.,  8.,  5.,  3.,  1.],
       [ 4.,  6., 10., 13., 16., 17.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.]])

In [21]:
# this is a single problem y, as described above
MoonBoard_2016_raw_forseq['Y_dict_benchmark_withgrade'][next(iter(MoonBoard_2016_raw_forseq['Y_dict_benchmark_withgrade']))]

array([[5],
       [1],
       [5]], dtype=int64)

Merge problem dicts

In [23]:
X_dict_merge = {**MoonBoard_2016_raw_forseq['X_dict_benchmark_withgrade'], 
                **MoonBoard_2016_raw_forseq['X_dict_benchmark_nograde'], 
                **MoonBoard_2016_raw_forseq['X_dict_withgrade'], 
                **MoonBoard_2016_raw_forseq['X_dict_nograde']}
Y_dict_merge = {**MoonBoard_2016_raw_forseq['Y_dict_benchmark_withgrade'], 
                **MoonBoard_2016_raw_forseq['Y_dict_benchmark_nograde'], 
                **MoonBoard_2016_raw_forseq['Y_dict_withgrade'], 
                **MoonBoard_2016_raw_forseq['Y_dict_nograde']}

Use Beamer search to calculate theoretically best route (sequence of LH/RH moves) for each problem

In [33]:
save_path_seq = constants.PREPROCESSING_DIR / 'processed_data_seq'
# TODO: currently broken, sequence data no longer contains successScoreSequence?
output_seq = helper.generate_organized_sequence_data(raw_data = X_dict_merge, save_path = save_path_seq)

data with key 337509 contains error


AttributeError: 'beta' object has no attribute 'successScoreSequence'

In [27]:
sequence = helper.load_pickle(save_path_seq)

Move sequence data is of shape `4 x num_moves`:
- x (start of move or end?)
- y
- -1/1 for LH/RH
- sequence success score

In [29]:
# TODO: does move (x,y) correspond to start or end of move?
sequence['X_dict_seq'][next(iter(sequence['X_dict_seq']))]

array([[10.        , 10.        ,  5.        ,  8.        ,  5.        ,
         3.        ,  1.        ],
       [ 4.        ,  4.        ,  6.        , 10.        , 13.        ,
        16.        , 17.        ],
       [-1.        ,  1.        , -1.        ,  1.        , -1.        ,
         1.        , -1.        ],
       [13.96661017,  3.77127226,  3.46223915,  4.29652958,  1.02261563,
         9.826095  , 40.8232162 ]])